In [1]:
from bs4 import BeautifulSoup
import urllib
import re 

In [146]:
def search(query):
    address = "http://www.bing.com/search?q=%s" % (urllib.parse.quote_plus(query)+ "&count=20")

    getRequest = urllib.request.Request(address, None, {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'})

    urlfile = urllib.request.urlopen(getRequest)
    htmlResult = urlfile.read(20000000)
    urlfile.close()

    soup = BeautifulSoup(htmlResult).find('div',id='b_content')

    [s.extract() for s in soup('span')]
    unwantedTags = ['cite']
    
    for tag in unwantedTags:
        for match in soup.findAll(tag):
            match.replaceWithChildren()

    results = soup.findAll('li')
    

    return results

def get_item_info(query):
    links = search(query)
    titles = []
    urls = []
    paragraphs = []
    for link in links :
        try:
            urls.append(str(link.find('h2').find('a')['href']).replace(" ", " "))
            titles.append(str(link.find('h2').text).replace(" ", " "))
        except:
            ""
        try:
            paragraphs.append(str(link.findAll('p')[0].text))
        except:
            paragraphs.append("")

    return titles, urls, paragraphs, links[0]

In [159]:
def get_water_data(item):
    titles, urls, paragraphs, b = get_item_info("how much do I need to water my "+item+" plant")
    i = 0
    for p in paragraphs:
        nums = parse_water_paragraph(p)
        if len(nums)==2:
            print(nums)
            return nums
    return [1,10]

def parse_water_paragraph(p):
    tmp = ""
    if 'inch' in p:
        tmp = p[(p.index('inch')-15):(p.index('inch'))]
    
    nums = re.findall('\d*\.?\d+',tmp)
    print(nums)
    return nums

def get_growth_data(item):
    results = search("how long does it take for a "+item+" to grow in days")
    ans = []
    for r in results:
        ans = parse_growth_paragraph(str(r))
        if len(ans)>0:
            return ans

    return ans

def parse_growth_paragraph(p):
    tmp = ""
    month = False
    if 'day' in p:
        print("Hello")
        tmp = p[(p.index('day')-15):(p.index('day'))]

    nums = re.findall('\d*\.?\d+',tmp)
            
    return nums

def get_expiration_data(item):
    results = search("how long does it take for a "+item+" to expire")
    ans = []
    for r in results:
        ans = parse_expiration_paragraph(str(r))
        if len(ans)>0:
            return ans

    return ans

def parse_expiration_paragraph(p):
    tmp = ""
    month = False
    if 'day' in p:
        print("Hello")
        tmp = p[(p.index('day')-15):(p.index('day'))]
    elif 'week' in p:
        tmp = "7"

    nums = re.findall('\d*\.?\d+',tmp)
            
    return nums


def get_shop_data(item):
    shopdata = shop("Buy "+item+" seeds")
    shopdata = []
    for l in links:
        data = {}
        try:
            data['image']='https://www.bing.com/'+l.findAll('img')[0]['src']
            data['seller'] = l.findAll('span', class_='br-sellersCite')[0].text
            data['price'] = l.findAll('div', class_='pd-price br-standardPrice promoted')[0].text
            try:
                data['item'] = l.findAll('span')[-1]['title']
            except:
                data['item'] = "shoes"
            data['url'] = l.findAll('a', class_='br-adClickOut')[0]['href']

            shopdata.append(data)
        except:
            print('DIDNT WORK')
            break
    return shopdata
    

In [148]:
get_water_data("daisy")

[]
['1', '2']
['1', '2']


['1', '2']

In [164]:
get_expiration_data('potato')

['7']